In [1]:
# 文件设置
INPUT_FILE = "001.mp4" # 输入文件名

# 这是将导出到Google Drive中“我的云端硬盘”下的“RVE”文件夹的输出文件。
# 将覆盖任何现有文件。
OUTPUT_FILE = "output.mkv" # 推荐使用mkv格式，因为它支持更多编解码器 {mkv, mp4, avi, mov}

KILL_RUNTIME_AFTER_RENDER = True # 渲染完成后终止运行时，以节省Colab计算资源。如果复制到云端硬盘失败，需要手动干预下载文件。
# 插帧设置

# https://github.com/TNTwise/real-video-enhancer-models/releases/tag/models/ 是RVE所有模型文件的存储位置
# 您可以从那里下载，也可以尝试 https://openmodeldb.info/ 的自定义模型
INTERPOLATE = True # {True, False} 是否启用插帧
INTERPOLATE_FACTOR = 2 # 插帧倍数
SCENE_DETECT_METHOD = "pyscenedetect" # {pyscenedetect, none} 场景检测方法
SCENE_DETECT_SENSATIVITY = "3.5" # {0 - 9.9} 场景检测灵敏度，数字越低越灵敏

"""
内部插帧模型:
rife4.6.pkl - 速度优先
rife4.7.pkl - 平滑度优先
rife4.18.pkl - 真实感
rife4.22.pkl - 动画
rife4.25.pkl - 通用
GMFSS.pkl - 动画
GMFSS_PRO.pkl - 动画
GIMMVFI_RAFT.pth - 真实感
"""
INTERPOLATE_MODEL = "rife4.25.pkl" # {内部模型, (模型路径链接)}


# 超分设置
UPSCALE = True # {True, False} 是否启用超分
"""
4xNomos8k_span_otf_weak.pth - 逼真 4倍 高质量输入
4xNomos8k_span_otf_medium.pth - 逼真 4倍 中等质量输入
4xNomos8k_span_otf_strong.pth - 逼真 4倍 低质量输入
2x_BHI_SpanPlusDynamic_Light.pth - 逼真 2倍 高质量输入 (仅pytorch，实验性)

2x_ModernSpanimationV2.pth - 动画 2倍
2x_ModernSpanimationV3.pth - 动画 2倍 (仅pytorch，实验性)
2x_AnimeJaNai_V2_Compact_36k.pth - 动画 2倍
2x_AnimeJaNai_HD_V3_Sharp1_Compact_430k.pth - 动画 2倍
up2x-latest-conservative.pth - 动画 2倍 (仅pytorch，慢)
up3x-latest-conservative.pth - 动画 3倍 (仅pytorch，慢)
up4x-latest-conservative.pth - 动画 4倍 (仅pytorch，慢)
"""
UPSCALE_MODEL = "2x_ModernSpanimationV2.pth" # {(模型路径链接)}

# FFMpeg设置
VIDEO_ENCODER = "libx264" # {libx264,libx265,vp9,av1,prores,ffv1,x264_vulkan,x264_nvenc,x265_nvenc} 视频编码器
AUDIO_ENCODER = "copy_audio" # {aac,libmp3lame,opus,copy_audio} 音频编码器
SUBTITLE_ENCODER = "copy_subtitle" # {srt,ass,webvtt,copy_subtitle} 字幕编码器

# 后端设置
BACKEND = "pytorch" # {pytorch, tensorrt} TensorRT速度最快，但安装依赖项需要很长时间。

In [2]:
# 完成后将输出文件移动到云端硬盘以进行永久存储
import os
import subprocess
import shutil
from google.colab import drive, output

# 挂载云端硬盘并将输出文件移动到那里以进行永久存储
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!rm -rf real-video-enhancer/
!git clone https://github.com/tntwise/real-video-enhancer --branch 2.3.8

Cloning into 'real-video-enhancer'...
remote: Enumerating objects: 27074, done.
remote: Counting objects: 100% (2737/2737), done.
remote: Compressing objects: 100% (2349/2349), done.
remote: Total 27074 (delta 427), reused 414 (delta 388), pack-reused 24337 (from 1)
Receiving objects: 100% (27074/27074), 22.03 MiB | 18.96 MiB/s, done.
Resolving deltas: 100% (20208/20208), done.


In [4]:
import os
!mkdir real-video-enhancer/models/ # 创建模型文件夹
origdir = os.getcwd() # 保存当前工作目录
os.chdir("real-video-enhancer/models/") # 切换到模型文件夹

def has_model(model: str):
    # 检查是否指定了模型
    return model and model.lower() != "none"

def download_model(model: str):
    # 从GitHub发布页面下载模型
    os.system(f"wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/{model}")



# 通过提取链接末尾来获取外部插帧模型
def download_external_model(model_link : str):
        print("Downloading Model")
        os.system(f"wget {model_link}")
        model = model_link.split(r'/')[-1]
        return model


if INTERPOLATE:
    # 如果启用插帧，则下载插帧模型
    if "http" in INTERPOLATE_MODEL:
        UPSCALE_MODEL = download_external_model(INTERPOLATE_MODEL)
    else:
        download_model(INTERPOLATE_MODEL)
if UPSCALE:
    # 如果启用超分，则下载超分模型
    if "http" in UPSCALE_MODEL:
        print("up")
        UPSCALE_MODEL = download_external_model(UPSCALE_MODEL)
    else:
        download_model(UPSCALE_MODEL)


os.chdir(origdir) # 恢复原始工作目录

In [5]:
!mkdir backend # create backend dir so rve doesnt crash. This is to prevent rve from not detecting a "backend" folder, and being unable to write to a non existant log file location.

In [6]:
if BACKEND.lower() == 'tensorrt':
    !wget https://github.com/astral-sh/python-build-standalone/releases/download/20250317/cpython-3.12.9+20250317-x86_64-unknown-linux-gnu-install_only.tar.gz
    !tar xvf cpython-3.12.9+20250317-x86_64-unknown-linux-gnu-install_only.tar.gz
    !./python/bin/python3 -m pip install  --extra-index-url https://download.pytorch.org/whl/cu126 -r real-video-enhancer/backend/requirements.txt # only install if using tensorrt

!mkdir bin/ && cd bin && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/ffmpeg && chmod +x ffmpeg

--2025-11-14 13:25:14--  https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/ffmpeg
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/837362765/87cad8f2-ed9d-4334-a81c-096918b3aac8?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-14T14%3A18%3A28Z&rscd=attachment%3B+filename%3Dffmpeg&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-14T13%3A18%3A22Z&ske=2025-11-14T14%3A18%3A28Z&sks=b&skv=2018-11-09&sig=wvisSoABftoI6khHMI6j4HNuAENSfiwu90KppZlh7ss%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MzEzMDMxNCwibmJmIjoxNzYzMTI2NzE0LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG9iL

In [7]:
# 示例超分/降噪命令
# 推荐使用 --video_encoder_preset x264_nvenc
def build_backend_command() -> str:
    command = (('python3 real-video-enhancer/backend/rve-backend.py' if BACKEND == "pytorch" else './python/bin/python3 real-video-enhancer/backend/rve-backend.py')
               + f' -i "{INPUT_FILE}"'
               + f' -o "{OUTPUT_FILE}"'
               + f' -b {BACKEND} '
               + f' --video_encoder_preset {VIDEO_ENCODER}'
               + f' --audio_encoder_preset {AUDIO_ENCODER}'
               + f' --subtitle_encoder_preset {SUBTITLE_ENCODER}')
    if UPSCALE_MODEL and UPSCALE:
        command += f" --upscale_model real-video-enhancer/models/{UPSCALE_MODEL} " # 添加超分模型参数
    if INTERPOLATE_MODEL and INTERPOLATE:
        command += f" --interpolate_model real-video-enhancer/models/{INTERPOLATE_MODEL} " # 添加插帧模型参数
        command += f" --interpolate_factor {INTERPOLATE_FACTOR} " # 添加插帧倍数参数
        command += f" --scene_detect_method {SCENE_DETECT_METHOD} " # 添加场景检测方法参数
        command += f" --scene_detect_threshold {SCENE_DETECT_SENSATIVITY} " # 添加场景检测灵敏度参数

    return command

with open("command.sh", "w") as f:
  f.write(build_backend_command()) # 将命令写入shell脚本文件

!bash command.sh # 执行shell脚本

BACKEND: Getting Input Video Properties
BACKEND: No video stream found in the input file.
Traceback (most recent call last):
  File "/content/real-video-enhancer/backend/rve-backend.py", line 515, in <module>
    HandleApplication()
  File "/content/real-video-enhancer/backend/rve-backend.py", line 30, in __init__
    print_video_info(video_info)
  File "/content/real-video-enhancer/backend/src/utils/VideoInfo.py", line 317, in print_video_info
    print(f"Duration: {video_info.get_duration_seconds()} seconds")
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/real-video-enhancer/backend/src/utils/VideoInfo.py", line 261, in get_duration_seconds
    duration = self.cap.get(cv2.CAP_PROP_FRAME_COUNT) / self.get_fps()
               ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~
ZeroDivisionError: float division by zero


In [ ]:
# Colab中的源文件路径
source_path = f'/content/{OUTPUT_FILE}'

# Google Drive（根目录）中的目标路径
destination_path = f'/content/drive/My Drive/RVE/{OUTPUT_FILE}'
# 如果目标文件夹不存在，则创建
if not os.path.exists(os.path.dirname(destination_path)):
  os.mkdir(os.path.dirname(destination_path))

# 移动文件
try:
  shutil.copy(source_path, destination_path) # 尝试将文件从源路径复制到目标路径
except Exception as e:
  print(f"复制到云端硬盘失败！{e}\n请在终止运行时之前下载所有已正确渲染的文件！") # 打印错误信息
  input("按回车键终止运行时。") # 等待用户输入以终止运行时

  output.eval_js('google.colab.kernel.disconnect()') # 断开Colab运行时连接
  os.kill(os.getpid(), 9) # 终止当前进程

if KILL_RUNTIME_AFTER_RENDER:
  # 终止运行时
  output.eval_js('google.colab.kernel.disconnect()') # 断开Colab运行时连接
  os.kill(os.getpid(), 9) # 终止当前进程

Copying to drive failed! [Errno 2] No such file or directory: '/content/output.mkv'
Please download any files that have correctly rendered before terminating the runtime!.
